In [11]:
import pandas as pd
import requests
import gmaps
import hvplot.pandas
import cartopy
import geoviews
import pyproj
import ipywidgets as widgets
from api_keys import geoapify_key, gmaps_key

In [12]:
# Load the CSV file with weather and coordinates data for each city
city_data_df = pd.read_csv("output_data/cities.csv")

# Set up gmaps with your API key
gmaps.configure(geoapify_key)

In [13]:
# Step 1: Create a map displaying a point for every city with the size of the point determined by humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(20, 0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

In [14]:
# Step 2: Narrow down the DataFrame to find cities with ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

In [15]:
# Step 3: Create a new DataFrame to store the cities with ideal weather conditions
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

In [16]:
# Step 4: Use the Geoapify API to find hotels for each city
hotel_name = []
hotel_country = []

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    url = f"https://api.geoapify.com/v1/places?categories=hotels&filter=distance,latlng:{lat},{lng}&limit=1&apiKey={geoapify_key}"
    response = requests.get(url).json()

    if response["features"]:
        hotel_name.append(response["features"][0]["properties"]["name"])
        hotel_country.append(response["features"][0]["properties"]["country"])
    else:
        hotel_name.append("N/A")
        hotel_country.append("N/A")

hotel_df["Hotel Name"] = hotel_name
hotel_df["Hotel Country"] = hotel_country

In [17]:
# Step 5: Create a map showing the hotels with the ideal weather conditions
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Hotel Country}</dd>
<dt>Humidity</dt><dd>{Humidity}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [18]:
map_plot=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
    size="Humidity",
    tiles="OSM"
)
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)